In [4]:
!pip install backoff

In [2]:
!git clone https://github.com/solpaul/fpl-prediction.git

Cloning into 'fpl-prediction'...
remote: Enumerating objects: 85615, done.
remote: Counting objects: 100% (10077/10077), done.
remote: Compressing objects: 100% (6166/6166), done.
remote: Total 85615 (delta 3834), reused 8897 (delta 3335), pack-reused 75538
Receiving objects: 100% (85615/85615), 135.00 MiB | 21.52 MiB/s, done.
Resolving deltas: 100% (39505/39505), done.
Checking out files: 100% (6809/6809), done.


In [5]:
%cd fpl-prediction/

[Errno 2] No such file or directory: 'fpl-prediction/'
/content/fpl-prediction


In [6]:
from pathlib import Path
import pandas as pd
from fpl_predictor.util import *

In [ ]:
# path to project direction
path = Path('./')

In [ ]:
train_df = pd.read_csv(path/'fpl_predictor/data/train_v8.csv',
                       index_col=0,
                       dtype={'season':str,
                              'squad':str,
                              'comp':str})

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114801 entries, 0 to 114802
Data columns (total 59 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   player                                      114801 non-null  object 
 1   gw                                          114801 non-null  int64  
 2   position                                    114801 non-null  int64  
 3   minutes                                     114801 non-null  int64  
 4   team                                        114801 non-null  object 
 5   opponent_team                               114801 non-null  object 
 6   relative_market_value_team                  46738 non-null   float64
 7   relative_market_value_opponent_team         46715 non-null   float64
 8   was_home                                    114801 non-null  bool   
 9   total_points                                114801 non-null  int64  
 

In [ ]:
train_df.head(10)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,assists,bonus,bps,clean_sheets,creativity,goals_conceded,goals_scored,ict_index,influence,own_goals,penalties_missed,penalties_saved,red_cards,saves,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba,relative_market_value_team_season,relative_market_value_opponent_team_season,date,squad,comp,shots_total,shots_on_target,touches,pressures,tackles,interceptions,blocks,xg,npxg,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
0,Aaron Cresswell,1,2,0,West Ham United,Chelsea,NaN,NaN,False,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0,0,14023,1,2,0.0,0,0,0,0,2016-08-15T19:00:00Z,1617,NaN,0.895471,2.243698,2016-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,Aaron Lennon,1,3,15,Everton,Tottenham Hotspur,NaN,NaN,True,1,0,0,6,0,0.3,0,0,0.9,8.2,0,0,0,0,0,13918,1,1,0.0,0,0,0,0,2016-08-13T14:00:00Z,1617,NaN,1.057509,1.433690,2016-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,Aaron Ramsey,1,3,60,Arsenal,Liverpool,NaN,NaN,True,2,0,0,5,0,4.9,3,0,3.0,2.2,0,0,0,0,0,163170,4,3,23.0,0,0,0,0,2016-08-14T15:00:00Z,1617,NaN,1.944129,1.465860,2016-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3,Abdoulaye Doucouré,1,3,0,Watford,Southampton,NaN,NaN,False,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0,0,1051,1,1,0.0,0,0,0,0,2016-08-13T14:00:00Z,1617,NaN,0.704200,0.796805,2016-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,Abdul Rahman Baba,1,2,0,Chelsea,West Ham United,NaN,NaN,True,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0,0,1243,1,2,0.0,0,0,0,0,2016-08-15T19:00:00Z,1617,NaN,2.243698,0.895471,2016-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
5,Abel Hernández,1,4,90,Hull City,Leicester City,NaN,NaN,True,5,1,0,10,0,12.2,1,0,5.7,14.4,0,0,0,0,0,26039,1,2,30.0,0,0,0,0,2016-08-13T11:30:00Z,1617,NaN,0.494447,0.650832,2016-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
6,Adama Diomande,1,4,90,Hull City,Leicester City,NaN,NaN,True,8,0,2,29,0,16.8,1,1,10.7,45.2,0,0,0,0,0,38151,1,2,45.0,0,0,0,0,2016-08-13T11:30:00Z,1617,NaN,0.494447,0.650832,2016-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
7,Adam Clayton,1,3,90,Middlesbrough,Stoke City,NaN,NaN,True,2,0,0,6,0,2.2,1,0,1.4,3.2,0,0,0,0,0,17663,1,1,9.0,0,0,0,0,2016-08-13T14:00:00Z,1617,NaN,0.452793,0.718705,2016-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
8,Adam Federici,1,1,0,Bournemouth,Manchester United,NaN,NaN,True,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0,0,4315,3,1,0.0,0,0,0,0,2016-08-14T12:30:00Z,1617,NaN,0.384921,1.983179,2016-08-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
9,Adam Forshaw,1,3,69,Middlesbrough,Stoke City,NaN,NaN,True,1,0,0,3,0,1.3,1,0,0.3,2.0,0,0,0,0,0,2723,1,1,0.0,0,0,0,1,2016-08-13T14:00:00Z,1617,NaN,0.452793,0.718705,2016-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [ ]:
train_df[train_df['season'] == '1718'].head(10)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,assists,bonus,bps,clean_sheets,creativity,goals_conceded,goals_scored,ict_index,influence,own_goals,penalties_missed,penalties_saved,red_cards,saves,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba,relative_market_value_team_season,relative_market_value_opponent_team_season,date,squad,comp,shots_total,shots_on_target,touches,pressures,tackles,interceptions,blocks,xg,npxg,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
23679,Aaron Cresswell,1,2,9,West Ham United,Manchester United,NaN,NaN,False,0,0,0,3,0,0.6,2,0,1.9,0.4,0,0,0,0,0,25136,0,4,18.0,0,0,0,0,2017-08-13T15:00:00Z,1718,NaN,0.866330,2.110135,2017-08-13,West Ham,Premier League,1.0,0.0,15.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,9.0,12.0,75.0,11.0,0.0,0.0,True
23680,Aaron Lennon,1,3,0,Everton,Stoke City,NaN,NaN,True,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0,0,4681,0,1,0.0,0,0,0,0,2017-08-12T14:00:00Z,1718,NaN,1.134226,0.581587,2017-08-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
23681,Aaron Mooy,1,3,90,Huddersfield Town,Crystal Palace,NaN,NaN,False,6,1,0,22,1,46.9,0,0,8.7,40.2,0,0,0,0,0,59955,3,0,0.0,0,0,0,0,2017-08-12T14:00:00Z,1718,NaN,0.210654,0.635984,2017-08-12,Huddersfield,Premier League,0.0,0.0,71.0,11.0,2.0,2.0,1.0,0.0,0.0,0.5,4.0,1.0,45.0,64.0,70.3,44.0,1.0,1.0,True
23682,Aaron Ramsey,1,3,23,Arsenal,Leicester City,NaN,NaN,True,6,0,0,16,0,11.2,0,1,6.7,29.6,0,0,0,0,0,33792,3,4,26.0,0,0,0,0,2017-08-11T18:45:00Z,1718,NaN,2.073500,0.824624,2017-08-11,Arsenal,Premier League,4.0,1.0,19.0,6.0,0.0,0.0,1.0,0.3,0.3,0.2,2.0,0.0,9.0,11.0,81.8,13.0,0.0,0.0,True
23683,Abdoulaye Doucouré,1,3,90,Watford,Liverpool,NaN,NaN,True,9,0,2,36,0,25.2,3,1,10.9,48.6,0,0,0,0,0,1207,3,3,35.0,0,0,0,0,2017-08-12T11:30:00Z,1718,NaN,0.547242,1.619155,2017-08-12,Watford,Premier League,1.0,1.0,79.0,12.0,1.0,1.0,2.0,0.6,0.6,0.1,3.0,0.0,59.0,70.0,84.3,54.0,2.0,2.0,True
23684,Adam Federici,1,1,0,Bournemouth,West Bromwich Albion,NaN,NaN,False,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0,0,224542,0,1,0.0,0,0,0,0,2017-08-12T14:00:00Z,1718,NaN,0.379765,0.541354,2017-08-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
23685,Adam Lallana,1,3,0,Liverpool,Watford,NaN,NaN,False,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0,0,45019,3,3,0.0,0,0,0,0,2017-08-12T11:30:00Z,1718,NaN,1.619155,0.547242,2017-08-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
23686,Adam Smith,1,2,10,Bournemouth,West Bromwich Albion,NaN,NaN,False,1,0,0,2,0,0.6,0,0,0.3,2.2,0,0,0,0,0,50168,0,1,0.0,0,0,0,0,2017-08-12T14:00:00Z,1718,NaN,0.379765,0.541354,2017-08-12,Bournemouth,Premier League,0.0,0.0,7.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,6.0,66.7,6.0,0.0,0.0,True
23687,Ademola Lookman,1,3,0,Everton,Stoke City,NaN,NaN,True,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0,0,8034,0,1,0.0,0,0,0,0,2017-08-12T14:00:00Z,1718,NaN,1.134226,0.581587,2017-08-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
23688,Adrian Mariappa,1,2,0,Watford,Liverpool,NaN,NaN,True,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0,0,0,0,171365,3,3,0.0,0,0,0,0,2017-08-12T11:30:00Z,1718,NaN,0.547242,1.619155,2017-08-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [ ]:
lag_train_df, team_lag_vars = team_lag_features(train_df, ['total_points'], ['all', 1, 2, 3])

In [ ]:
lag_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114801 entries, 0 to 114800
Data columns (total 95 columns):
 #   Column                                           Non-Null Count   Dtype  
---  ------                                           --------------   -----  
 0   player                                           114801 non-null  object 
 1   gw                                               114801 non-null  int64  
 2   position                                         114801 non-null  int64  
 3   minutes                                          114801 non-null  int64  
 4   team                                             114801 non-null  object 
 5   opponent_team                                    114801 non-null  object 
 6   relative_market_value_team                       46738 non-null   float64
 7   relative_market_value_opponent_team              46715 non-null   float64
 8   was_home                                         114801 non-null  bool   
 9   total_points   

In [ ]:
lag_train_df[(lag_train_df['season'] == '1718') & (lag_train_df['player'] == 'Harry Kane')].head(10)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,assists,bonus,bps,clean_sheets,creativity,goals_conceded,goals_scored,ict_index,influence,own_goals,penalties_missed,penalties_saved,red_cards,saves,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba,relative_market_value_team_season,relative_market_value_opponent_team_season,date,squad,comp,...,carries,dribbles_completed,dribbles,crowds,total_points_team,total_points_team_conceded,total_points_team_last_all,total_points_team_conceded_last_all,total_points_team_pg_last_all,total_points_team_conceded_pg_last_all,total_points_team_last_1,total_points_team_conceded_last_1,total_points_team_pg_last_1,total_points_team_conceded_pg_last_1,total_points_team_last_2,total_points_team_conceded_last_2,total_points_team_pg_last_2,total_points_team_conceded_pg_last_2,total_points_team_last_3,total_points_team_conceded_last_3,total_points_team_pg_last_3,total_points_team_conceded_pg_last_3,total_points_team_opponent,total_points_team_conceded_opponent,total_points_team_last_all_opponent,total_points_team_conceded_last_all_opponent,total_points_team_pg_last_all_opponent,total_points_team_conceded_pg_last_all_opponent,total_points_team_last_1_opponent,total_points_team_conceded_last_1_opponent,total_points_team_pg_last_1_opponent,total_points_team_conceded_pg_last_1_opponent,total_points_team_last_2_opponent,total_points_team_conceded_last_2_opponent,total_points_team_pg_last_2_opponent,total_points_team_conceded_pg_last_2_opponent,total_points_team_last_3_opponent,total_points_team_conceded_last_3_opponent,total_points_team_pg_last_3_opponent,total_points_team_conceded_pg_last_3_opponent
23863,Harry Kane,1,4,90,Tottenham Hotspur,Newcastle United,NaN,NaN,False,1,0,0,-6,1,30.6,0,0,12.2,4.0,0,0,0,0,0,1473220,2,0,87.0,0,0,0,1,2017-08-13T12:30:00Z,1718,NaN,1.751141,0.547176,2017-08-13,Tottenham,Premier League,...,15.0,1.0,4.0,True,71,15,2062,1110,54.263158,29.210526,88.0,18.0,88.0,18.0,163.0,37.0,81.5,18.5,208.0,63.0,69.333333,21.000000,15,71,0,0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
24375,Harry Kane,2,4,90,Tottenham Hotspur,Chelsea,NaN,NaN,True,1,0,0,-3,0,23.0,2,0,11.8,7.6,0,0,0,0,0,1429411,2,1,87.0,-151094,13666,164760,1,2017-08-20T15:00:00Z,1718,NaN,1.751141,2.125018,2017-08-20,Tottenham,Premier League,...,30.0,4.0,8.0,True,22,45,2133,1125,54.692308,28.846154,71.0,15.0,71.0,15.0,159.0,33.0,79.5,16.5,234.0,52.0,78.000000,17.333333,45,22,2087,1113,53.512821,28.538462,26.0,43.0,26.0,43.0,95.0,66.0,47.5,33.0,149.0,92.0,49.666667,30.666667
24898,Harry Kane,3,4,90,Tottenham Hotspur,Burnley,NaN,NaN,True,2,0,0,7,0,15.3,1,0,15.2,7.8,0,0,0,0,0,1390600,1,1,129.0,-67120,71103,138223,0,2017-08-27T15:00:00Z,1718,NaN,1.751141,0.316798,2017-08-27,Tottenham,Premier League,...,20.0,1.0,1.0,True,34,35,2155,1170,53.875000,29.250000,22.0,45.0,22.0,45.0,93.0,60.0,46.5,30.0,181.0,78.0,60.333333,26.000000,35,34,1452,1733,36.300000,43.325000,23.0,57.0,23.0,57.0,66.0,83.0,33.0,41.5,93.0,125.0,31.000000,41.666667
25433,Harry Kane,4,4,84,Tottenham Hotspur,Everton,NaN,NaN,False,13,0,3,58,1,15.3,0,2,15.2,67.8,0,0,0,0,0,1269420,3,0,69.0,-140803,51894,192697,0,2017-09-09T14:00:00Z,1718,NaN,1.751141,1.134226,2017-09-09,Tottenham,Premier League,...,25.0,1.0,3.0,True,80,16,2189,1205,53.390244,29.390244,34.0,35.0,34.0,35.0,56.0,80.0,28.0,40.0,127.0,95.0,42.333333,31.666667,16,80,1837,1535,44.804878,37.439024,17.0,73.0,17.0,73.0,56.0,97.0,28.0,48.5,109.0,121.0,36.333333,40.333333
25979,Harry Kane,5,4,90,Tottenham Hotspur,Swansea City,NaN,NaN,True,2,0,0,1,1,3.7,0,0,4.0,2.8,0,0,0,0,0,1499862,0,0,33.0,207021,222122,15101,0,2017-09-16T16:30:00Z,1718,NaN,1.751141,0.560457,2017-09-16,Tottenham,Premier League,...,22.0,1.0,1.0,True,50,54,2269,1221,54.023810,29.071429,80.0,16.0,80.0,16.0,114.0,51.0,57.0,25.5,136.0,96.0,45.333333,32.000000,54,50,1542,1938,36.71428

In [ ]:
lag_train_df, player_lag_vars = player_lag_features(lag_train_df, ['total_points'], ['all', 1, 2, 3])

In [ ]:
pd.options.display.max_columns = None

In [ ]:
lag_train_df[(lag_train_df['season'] == '1718') & (lag_train_df['player'] == 'Harry Kane')].head(10)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,assists,bonus,bps,clean_sheets,creativity,goals_conceded,goals_scored,ict_index,influence,own_goals,penalties_missed,penalties_saved,red_cards,saves,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba,relative_market_value_team_season,relative_market_value_opponent_team_season,date,squad,comp,shots_total,shots_on_target,touches,pressures,tackles,interceptions,blocks,xg,npxg,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds,total_points_team,total_points_team_conceded,total_points_team_last_all,total_points_team_conceded_last_all,total_points_team_pg_last_all,total_points_team_conceded_pg_last_all,total_points_team_last_1,total_points_team_conceded_last_1,total_points_team_pg_last_1,total_points_team_conceded_pg_last_1,total_points_team_last_2,total_points_team_conceded_last_2,total_points_team_pg_last_2,total_points_team_conceded_pg_last_2,total_points_team_last_3,total_points_team_conceded_last_3,total_points_team_pg_last_3,total_points_team_conceded_pg_last_3,total_points_team_opponent,total_points_team_conceded_opponent,total_points_team_last_all_opponent,total_points_team_conceded_last_all_opponent,total_points_team_pg_last_all_opponent,total_points_team_conceded_pg_last_all_opponent,total_points_team_last_1_opponent,total_points_team_conceded_last_1_opponent,total_points_team_pg_last_1_opponent,total_points_team_conceded_pg_last_1_opponent,total_points_team_last_2_opponent,total_points_team_conceded_last_2_opponent,total_points_team_pg_last_2_opponent,total_points_team_conceded_pg_last_2_opponent,total_points_team_last_3_opponent,total_points_team_conceded_last_3_opponent,total_points_team_pg_last_3_opponent,total_points_team_conceded_pg_last_3_opponent,minutes_last_all,minutes_last_1,minutes_last_2,minutes_last_3,total_points_last_all,total_points_pg_last_all,total_points_last_1,total_points_pg_last_1,total_points_last_2,total_points_pg_last_2,total_points_last_3,total_points_pg_last_3
23863,Harry Kane,1,4,90,Tottenham Hotspur,Newcastle United,NaN,NaN,False,1,0,0,-6,1,30.6,0,0,12.2,4.0,0,0,0,0,0,1473220,2,0,87.0,0,0,0,1,2017-08-13T12:30:00Z,1718,NaN,1.751141,0.547176,2017-08-13,Tottenham,Premier League,6.0,1.0,29.0,13.0,0.0,0.0,0.0,1.2,1.2,0.1,5.0,2.0,15.0,16.0,93.8,15.0,1.0,4.0,True,71,15,2062,1110,54.263158,29.210526,88.0,18.0,88.0,18.0,163.0,37.0,81.5,18.5,208.0,63.0,69.333333,21.000000,15,71,0,0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,2523,78.0,168.0,258.0,224,7.990488,17.0,19.615385,41.0,21.964286,48.0,16.744186
24375,Harry Kane,2,4,90,Tottenham Hotspur,Chelsea,NaN,NaN,True,1,0,0,-3,0,23.0,2,0,11.8,7.6,0,0,0,0,0,1429411,2,1,87.0,-151094,13666,164760,1,2017-08-20T15:00:00Z,1718,NaN,1.751141,2.125018,2017-08-20,Tottenham,Premier League,6.0,2.0,42.0,7.0,1.0,0.0,1.0,0.3,0.3,0.1,2.0,0.0,14.0,23.0,60.9,30.0,4.0,8.0,True,22,45,2133,1125,54.692308,28.846154,71.0,15.0,71.0,15.0,159.0,33.0,79.5,16.5,234.0,52.0,78.000000,17.333333,45,22,2087,1113,53.512821,28.538462,26.0,43.0,26.0,43.0,95.0,66.0,47.5,33.0,149.0,92.0,49.666667,30.666667,2613,90.0,168.0,258.0,225,7.749713,1.0,1.000000,18.0,9.642857,42.0,14.651163
24898,Harry Kane,3,4,90,Tottenham Hotspur,Burnley,NaN,NaN,True,2,0,0,7,0,15.3,1,0,15.2,7.8,0,0,0,0,0,1390600,1,1,129.0,-67120,71103,138223,0,2017-08-27T15:00:00Z,1718,NaN,1.751141,0.316798,2017-08-27,Tottenham,Premier League,10.0,2.0,35.0,6.0,0.0,0.0,1.0,1.1,1.1,0.0,5.0,0.0,13.0,23.0,56.5,20.0,1.0,1.0,True,34,35,2155,1170,53.875000,29.250000,22.0,45.0,22.0,45.0,93.0,60.0,46.5,30.0,181.0,78.0,60.333333,26.000000,35,34,1452,1733,36.300000,43.325000,23.0,57.0,23.0,57.0,66.0,83.0,33.0,41.5,93.0,125.0,31.000000,41.666667,2703,90.0,180.0,258.0,226,7.524972,1.0,1.000000,2.0,1.000000,19.0,6.627907
25433,Harry Kane,4,4,84,Tottenham Hotspur,Everton,NaN,NaN,False,13,0,3

In [ ]:
lag_train_df.shape

(114801, 107)

In [ ]:
# historical points per 90 minutes for a player at a particular point in time
lag_train_df[(lag_train_df['season'] == '2021') & 
             (lag_train_df['gw'] == 38) & 
             (lag_train_df['player'] == 'Mohamed Salah')]['total_points_pg_last_all'].mean()

7.634095634095634

In [ ]:
# points per minute across all players and minutes
(train_df['total_points'].sum() * 90 / train_df['minutes'].sum())

3.746588626353545

In [ ]:
# extreme example of points per minute for all appearances under 10 minutes
(train_df[train_df['minutes'] < 10]['total_points'].sum() * 90 / train_df[train_df['minutes'] < 10]['minutes'].sum())

22.143577188940093

In [ ]:
train_df[['was_home', 'goals_scored']].groupby('was_home').sum('goals_scored')

,goals_scored
was_home,
False,2267
True,2775


In [ ]:
train_df[train_df['kickoff_time'] < '2020-03-15'][['was_home', 'goals_scored']].groupby('was_home').sum()

,goals_scored
was_home,
False,1670
True,2144


In [ ]:
train_df[train_df['kickoff_time'] > '2020-03-15'][['was_home', 'goals_scored']].groupby('was_home').sum()

,goals_scored
was_home,
False,597
True,631


In [ ]:
train_df[['crowds', 'was_home', 'total_points']].groupby(['crowds', 'was_home']).sum()

total_points
crowds was_home              
False  False            18586
       True             19056
True   False            54278
       True             64125

In [ ]:
grouped_points = train_df[['season', 'crowds', 'was_home', 'total_points']].groupby(['season', 'crowds', 'was_home']).sum()

grouped_points["percentage"] = grouped_points.groupby(['season', 'crowds']).apply(
     lambda x:  100*x / x.sum()
)

grouped_points

total_points  percentage
season crowds was_home                          
1617   True   False            14099   45.309638
              True             17018   54.690362
1718   True   False            14372   45.715376
              True             17066   54.284624
1819   True   False            14452   46.572782
              True             16579   53.427218
1920   False  False             3614   46.214834
              True              4206   53.785166
       True   False            10647   45.908072
              True             12545   54.091928
2021   False  False            14972   50.204547
              True             14850   49.795453
       True   False              708   43.569231
              True               917   56.430769